In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
import datetime
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
import random
import visualkeras
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-15 12:46:59.230369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 12:47:00.042046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [5]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_0_to_1)

In [3]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-09-15 12:47:03.132667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-15 12:47:03.228731: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-15 12:47:03.228796: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-15 12:47:03.231341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-15 12:47:03.231397: I tensorflow/compile

In [12]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20
765/765 [==============================] - 20s 25ms/step - loss: 1.2704 - accuracy: 0.4333 - val_loss: 1.0645 - val_accuracy: 0.5400
Epoch 2/20
765/765 [==============================] - 11s 14ms/step - loss: 1.1189 - accuracy: 0.5237 - val_loss: 0.9104 - val_accuracy: 0.6288
Epoch 3/20
765/765 [==============================] - 11s 14ms/step - loss: 0.9524 - accuracy: 0.6125 - val_loss: 0.7460 - val_accuracy: 0.6999
Epoch 4/20
765/765 [==============================] - 11s 14ms/step - loss: 0.8257 - accuracy: 0.6793 - val_loss: 0.6498 - val_accuracy: 0.7409
Epoch 5/20
765/765 [==============================] - 11s 14ms/step - loss: 0.7613 - accuracy: 0.7105 - val_loss: 0.6083 - val_accuracy: 0.7584
Epoch 6/20
765/765 [==============================] - 11s 14ms/step - loss: 0.7156 - accuracy: 0.7282 - val_loss: 0.5742 - val_accuracy: 0.7847
Epoch 7/20
765/765 [==============================] - 11s 14ms/step - loss: 0.6701 - accuracy: 0.7440 - val_loss: 0.5310 - val_accuracy:

In [15]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 147, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 30, 143, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 71, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 69, 128)       24704     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 34, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 32, 256)       98560 